<div align="center">
    <h1>
        <a href="https://kokorottsai.com/zh#tryonline">kokoro:轻量级文字转语音</a>
    </h1>
</div>

- 环境要求：
    ```yaml
    name: kokoro
        channels:
          - defaults
        dependencies:
          - python==3.9       
          - libstdcxx~=12.4.0 # Needed to load espeak correctly. Try removing this if you're facing issues with Espeak fallback. 
          - pip:
              - kokoro>=0.3.1
              - soundfile
              - misaki[en]
    ```

- 安装
  ```bash
    pip install kokoro soundfile
    pip install transformers==4.30.0 huggingface-hub==0.15.1
    pip install misaki[zh]
  ```

### 1. 加载kokoro模型
- 默认还是会去[hugging face](https://huggingface.co/hexgrad/Kokoro-82M-v1.1-zh/tree/main)上下载一遍pth文件
- 还是建议手动下载，放在对应文件夹就行 `git clone https://huggingface.co/hexgrad/Kokoro-82M-v1.1-zh`

In [10]:
from kokoro import KModel, KPipeline
from pathlib import Path
import numpy as np
import soundfile as sf
import torch
import tqdm
import json
from IPython.display import display, Audio

with open("Kokoro-82M-v1.1-zh/config.json", 'r') as f:
    config = json.load(f)
    

kmodel = KModel(
    repo_id='hexgrad/Kokoro-82M-v1.1-zh',
    config=config,
    model="Kokoro-82M-v1.1-zh/kokoro-v1_1-zh.pth"
    )

pipeline = KPipeline(lang_code='z',
                     repo_id='hexgrad/Kokoro-82M-v1.1-zh',
                     model=kmodel
                     )


/home/sumi/anaconda3/lib/python3.10/site-packages/torch/nn/modules/rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn(
/home/sumi/anaconda3/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


### 2. 定义文字，加载声线
- 声线模型在voice文件夹找

In [22]:

text = '华子hr叫我国内9点，我这里凌晨三点面试[捂脸]，我说我起不来[捂脸]'
voice_tensor = torch.load('Kokoro-82M-v1.1-zh/voices/zf_001.pt', weights_only=True)


### 3. 推理，输出结果，保存为音频

In [23]:

generator = pipeline(
    text, voice=voice_tensor,
    speed=1, split_pattern=r'\n+'
)

gs, ps, audio = next(generator)
print(gs)
print(ps)
display(Audio(data=audio, rate=24000, autoplay=True))
    

华子hr叫我国内9点，我这里凌晨三点面试[捂脸]，我说我起不来[捂脸]
ㄏ穵2ㄗㄭ5 ❓ ㄐ要4/我3/ㄍ我2ㄋㄟ4/ㄐ又2ㄉ言3, 我3/ㄓㄜ4ㄌㄧ3/ㄌ应2ㄔㄣ2/ㄙㄢ1ㄉ言3/ㄇ言4ㄕ十4❓ㄨ2ㄌ言3❓, 我3/ㄕ我1/我3/ㄑㄧ3ㄅㄨ5ㄌㄞ2❓ㄨ2ㄌ言3❓


In [24]:
for i, (gs, ps, audio) in enumerate(generator):
    print(i)  # i => index
    print(gs) # gs => graphemes/text
    print(ps) # ps => phonemes
    display(Audio(data=audio, rate=24000, autoplay=i==0))
    sf.write(f'{i}.wav', audio, 24000) # save each audio file
    
type(audio)

torch.Tensor